In [302]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity,linear_kernel
import tracemalloc
tracemalloc.start()

In [303]:
rec_system = pd.read_parquet("../datasets/rec_System.parquet").head(20000)
rec_system_copy = rec_system.copy()

In [304]:
rec_system

,title,overview,name_genre,actors_names,director_names,tagline,company,collection
0,Toy Story,"Led by Woody, Andy's toys live happily in his ...","[Comedy, Animation, Family]","[Sarah Freeman, Tim Allen, Annie Potts, Penn J...",[John Lasseter],None,[Pixar Animation Studios],Toy Story Collection
1,Jumanji,When siblings Judy and Peter discover an encha...,"[Fantasy, Adventure, Family]","[Malcolm Stewart, David Alan Grier, Bradley Pi...",[Joe Johnston],Roll the dice and unleash the excitement!,"[Interscope Communications, TriStar Pictures, ...",[]
2,Grumpier Old Men,A family wedding reignites the ancient feud be...,"[Romance, Comedy]","[Jack Lemmon, Burgess Meredith, Walter Matthau...",[Howard Deutch],Still Yelling. Still Fighting. Still Ready for...,"[Warner Bros., Lancaster Gate]",Grumpy Old Men Collection
3,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...","[Comedy, Drama, Romance]","[Angela Bassett, Mykelti Williamson, Lela Roch...",[Forest Whitaker],Friends are the people who let you be yourself...,[Twentieth Century Fox Film Corporation],[]
4,Father of the Bride Part II,Just when George Banks has recovered from his ...,[Comedy],"[Kimberly Williams-Paisley, Lori Alan, BD Wong...",[Charles Shyer],Just When His World Is Back To Normal... He's ...,"[Sandollar Productions, Touchstone Pictures]",Father of the Bride Collection
...,...,...,...,...,...,...,...,...
19995,"After Fall, Winter","WINTER is a dangerous, sexy, poignant and at t...","[Romance, Drama]","[Deborah Twiss, Lizzie Brocheré, Eric Schaeffe...",[Eric Schaeffer],None,None,[]
19996,Violeta Went to Heaven,A portrait of famed Chilean singer and folklor...,[Drama],"[Thomas Durand, Luis Machín, Patricio Ossa, Ch...",[Andrés Wood],Creation is a bird without a flight plan,[Instituto Nacional de Cine y Artes Audiovisua...,[]
19997,Rebellion,Dissidents in a French colony attack a police ...,"[Action, Drama, History]","[Malik Zidi, Daniel Martin, Jean-Philippe Puym...",[Mathieu Kassovitz],None,"[Studio 37, Nord-Ouest Productions, UGC Images...",[]
19998,Versailles,A young mother Nina and her son Enzo find them...,[Drama],"[Judith Chemla, Max Baissette de Malglaive, Ph...",[Pierre Schöller],None,"[Procirep, Media Programme of the European Com...",[]


In [305]:
rec_system_copy.isna().sum()

title                0
overview           133
name_genre         461
actors_names       561
director_names     158
tagline           8294
company           3455
collection           0
dtype: int64

In [306]:
rec_system.fillna({"overview":"[]",
                   "name_genre":"[]",
                   "actors_names":"[]",
                   "director_names":"[]",
                   "tagline":"[]",
                   "company":"[]",
                   },inplace=True)

* Transformo la columna overview a una lista con la funcion `split`

In [307]:
#NO
# rec_system["overview"] = rec_system["overview"].apply(lambda x: x.split())
# rec_system["tagline"] = rec_system["tagline"].apply(lambda x: x.split())

In [308]:
rec_system

,title,overview,name_genre,actors_names,director_names,tagline,company,collection
0,Toy Story,"Led by Woody, Andy's toys live happily in his ...","[Comedy, Animation, Family]","[Sarah Freeman, Tim Allen, Annie Potts, Penn J...",[John Lasseter],[],[Pixar Animation Studios],Toy Story Collection
1,Jumanji,When siblings Judy and Peter discover an encha...,"[Fantasy, Adventure, Family]","[Malcolm Stewart, David Alan Grier, Bradley Pi...",[Joe Johnston],Roll the dice and unleash the excitement!,"[Interscope Communications, TriStar Pictures, ...",[]
2,Grumpier Old Men,A family wedding reignites the ancient feud be...,"[Romance, Comedy]","[Jack Lemmon, Burgess Meredith, Walter Matthau...",[Howard Deutch],Still Yelling. Still Fighting. Still Ready for...,"[Warner Bros., Lancaster Gate]",Grumpy Old Men Collection
3,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...","[Comedy, Drama, Romance]","[Angela Bassett, Mykelti Williamson, Lela Roch...",[Forest Whitaker],Friends are the people who let you be yourself...,[Twentieth Century Fox Film Corporation],[]
4,Father of the Bride Part II,Just when George Banks has recovered from his ...,[Comedy],"[Kimberly Williams-Paisley, Lori Alan, BD Wong...",[Charles Shyer],Just When His World Is Back To Normal... He's ...,"[Sandollar Productions, Touchstone Pictures]",Father of the Bride Collection
...,...,...,...,...,...,...,...,...
19995,"After Fall, Winter","WINTER is a dangerous, sexy, poignant and at t...","[Romance, Drama]","[Deborah Twiss, Lizzie Brocheré, Eric Schaeffe...",[Eric Schaeffer],[],[],[]
19996,Violeta Went to Heaven,A portrait of famed Chilean singer and folklor...,[Drama],"[Thomas Durand, Luis Machín, Patricio Ossa, Ch...",[Andrés Wood],Creation is a bird without a flight plan,[Instituto Nacional de Cine y Artes Audiovisua...,[]
19997,Rebellion,Dissidents in a French colony attack a police ...,"[Action, Drama, History]","[Malik Zidi, Daniel Martin, Jean-Philippe Puym...",[Mathieu Kassovitz],[],"[Studio 37, Nord-Ouest Productions, UGC Images...",[]
19998,Versailles,A young mother Nina and her son Enzo find them...,[Drama],"[Judith Chemla, Max Baissette de Malglaive, Ph...",[Pierre Schöller],[],"[Procirep, Media Programme of the European Com...",[]


In [309]:
def collapse(valor):
    valores =[]
    for i in valor:
       valores.append(i)
    return valores

In [310]:
rec_system

,title,overview,name_genre,actors_names,director_names,tagline,company,collection
0,Toy Story,"Led by Woody, Andy's toys live happily in his ...","[Comedy, Animation, Family]","[Sarah Freeman, Tim Allen, Annie Potts, Penn J...",[John Lasseter],[],[Pixar Animation Studios],Toy Story Collection
1,Jumanji,When siblings Judy and Peter discover an encha...,"[Fantasy, Adventure, Family]","[Malcolm Stewart, David Alan Grier, Bradley Pi...",[Joe Johnston],Roll the dice and unleash the excitement!,"[Interscope Communications, TriStar Pictures, ...",[]
2,Grumpier Old Men,A family wedding reignites the ancient feud be...,"[Romance, Comedy]","[Jack Lemmon, Burgess Meredith, Walter Matthau...",[Howard Deutch],Still Yelling. Still Fighting. Still Ready for...,"[Warner Bros., Lancaster Gate]",Grumpy Old Men Collection
3,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...","[Comedy, Drama, Romance]","[Angela Bassett, Mykelti Williamson, Lela Roch...",[Forest Whitaker],Friends are the people who let you be yourself...,[Twentieth Century Fox Film Corporation],[]
4,Father of the Bride Part II,Just when George Banks has recovered from his ...,[Comedy],"[Kimberly Williams-Paisley, Lori Alan, BD Wong...",[Charles Shyer],Just When His World Is Back To Normal... He's ...,"[Sandollar Productions, Touchstone Pictures]",Father of the Bride Collection
...,...,...,...,...,...,...,...,...
19995,"After Fall, Winter","WINTER is a dangerous, sexy, poignant and at t...","[Romance, Drama]","[Deborah Twiss, Lizzie Brocheré, Eric Schaeffe...",[Eric Schaeffer],[],[],[]
19996,Violeta Went to Heaven,A portrait of famed Chilean singer and folklor...,[Drama],"[Thomas Durand, Luis Machín, Patricio Ossa, Ch...",[Andrés Wood],Creation is a bird without a flight plan,[Instituto Nacional de Cine y Artes Audiovisua...,[]
19997,Rebellion,Dissidents in a French colony attack a police ...,"[Action, Drama, History]","[Malik Zidi, Daniel Martin, Jean-Philippe Puym...",[Mathieu Kassovitz],[],"[Studio 37, Nord-Ouest Productions, UGC Images...",[]
19998,Versailles,A young mother Nina and her son Enzo find them...,[Drama],"[Judith Chemla, Max Baissette de Malglaive, Ph...",[Pierre Schöller],[],"[Procirep, Media Programme of the European Com...",[]


In [311]:
# Con parquet, los tipos list pasan a ser ndarray, con este metodo los volvemos lista
rec_system["name_genre"]=rec_system["name_genre"].apply(collapse)
rec_system["actors_names"]=rec_system["actors_names"].apply(collapse)
rec_system["director_names"]=rec_system["director_names"].apply(collapse)
rec_system["company"]=rec_system["company"].apply(collapse)
# rec_system["tagline"]=rec_system["tagline"].apply(collapse)

In [312]:
rec_system

,title,overview,name_genre,actors_names,director_names,tagline,company,collection
0,Toy Story,"Led by Woody, Andy's toys live happily in his ...","[Comedy, Animation, Family]","[Sarah Freeman, Tim Allen, Annie Potts, Penn J...",[John Lasseter],[],[Pixar Animation Studios],Toy Story Collection
1,Jumanji,When siblings Judy and Peter discover an encha...,"[Fantasy, Adventure, Family]","[Malcolm Stewart, David Alan Grier, Bradley Pi...",[Joe Johnston],Roll the dice and unleash the excitement!,"[Interscope Communications, TriStar Pictures, ...",[]
2,Grumpier Old Men,A family wedding reignites the ancient feud be...,"[Romance, Comedy]","[Jack Lemmon, Burgess Meredith, Walter Matthau...",[Howard Deutch],Still Yelling. Still Fighting. Still Ready for...,"[Warner Bros., Lancaster Gate]",Grumpy Old Men Collection
3,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...","[Comedy, Drama, Romance]","[Angela Bassett, Mykelti Williamson, Lela Roch...",[Forest Whitaker],Friends are the people who let you be yourself...,[Twentieth Century Fox Film Corporation],[]
4,Father of the Bride Part II,Just when George Banks has recovered from his ...,[Comedy],"[Kimberly Williams-Paisley, Lori Alan, BD Wong...",[Charles Shyer],Just When His World Is Back To Normal... He's ...,"[Sandollar Productions, Touchstone Pictures]",Father of the Bride Collection
...,...,...,...,...,...,...,...,...
19995,"After Fall, Winter","WINTER is a dangerous, sexy, poignant and at t...","[Romance, Drama]","[Deborah Twiss, Lizzie Brocheré, Eric Schaeffe...",[Eric Schaeffer],[],"[[, ]]",[]
19996,Violeta Went to Heaven,A portrait of famed Chilean singer and folklor...,[Drama],"[Thomas Durand, Luis Machín, Patricio Ossa, Ch...",[Andrés Wood],Creation is a bird without a flight plan,[Instituto Nacional de Cine y Artes Audiovisua...,[]
19997,Rebellion,Dissidents in a French colony attack a police ...,"[Action, Drama, History]","[Malik Zidi, Daniel Martin, Jean-Philippe Puym...",[Mathieu Kassovitz],[],"[Studio 37, Nord-Ouest Productions, UGC Images...",[]
19998,Versailles,A young mother Nina and her son Enzo find them...,[Drama],"[Judith Chemla, Max Baissette de Malglaive, Ph...",[Pierre Schöller],[],"[Procirep, Media Programme of the European Com...",[]


In [313]:
rec_system["name_genre"] = rec_system["name_genre"].apply(lambda x: ",".join(x))
rec_system["actors_names"] = rec_system["actors_names"].apply(lambda x: ",".join(x))
rec_system["director_names"] = rec_system["director_names"].apply(lambda x: ",".join(x))
rec_system["company"] = rec_system["company"].apply(lambda x: ",".join(x))

In [314]:
rec_system_copy

,title,overview,name_genre,actors_names,director_names,tagline,company,collection
0,Toy Story,"Led by Woody, Andy's toys live happily in his ...","[Comedy, Animation, Family]","[Sarah Freeman, Tim Allen, Annie Potts, Penn J...",[John Lasseter],None,[Pixar Animation Studios],Toy Story Collection
1,Jumanji,When siblings Judy and Peter discover an encha...,"[Fantasy, Adventure, Family]","[Malcolm Stewart, David Alan Grier, Bradley Pi...",[Joe Johnston],Roll the dice and unleash the excitement!,"[Interscope Communications, TriStar Pictures, ...",[]
2,Grumpier Old Men,A family wedding reignites the ancient feud be...,"[Romance, Comedy]","[Jack Lemmon, Burgess Meredith, Walter Matthau...",[Howard Deutch],Still Yelling. Still Fighting. Still Ready for...,"[Warner Bros., Lancaster Gate]",Grumpy Old Men Collection
3,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...","[Comedy, Drama, Romance]","[Angela Bassett, Mykelti Williamson, Lela Roch...",[Forest Whitaker],Friends are the people who let you be yourself...,[Twentieth Century Fox Film Corporation],[]
4,Father of the Bride Part II,Just when George Banks has recovered from his ...,[Comedy],"[Kimberly Williams-Paisley, Lori Alan, BD Wong...",[Charles Shyer],Just When His World Is Back To Normal... He's ...,"[Sandollar Productions, Touchstone Pictures]",Father of the Bride Collection
...,...,...,...,...,...,...,...,...
19995,"After Fall, Winter","WINTER is a dangerous, sexy, poignant and at t...","[Romance, Drama]","[Deborah Twiss, Lizzie Brocheré, Eric Schaeffe...",[Eric Schaeffer],None,None,[]
19996,Violeta Went to Heaven,A portrait of famed Chilean singer and folklor...,[Drama],"[Thomas Durand, Luis Machín, Patricio Ossa, Ch...",[Andrés Wood],Creation is a bird without a flight plan,[Instituto Nacional de Cine y Artes Audiovisua...,[]
19997,Rebellion,Dissidents in a French colony attack a police ...,"[Action, Drama, History]","[Malik Zidi, Daniel Martin, Jean-Philippe Puym...",[Mathieu Kassovitz],None,"[Studio 37, Nord-Ouest Productions, UGC Images...",[]
19998,Versailles,A young mother Nina and her son Enzo find them...,[Drama],"[Judith Chemla, Max Baissette de Malglaive, Ph...",[Pierre Schöller],None,"[Procirep, Media Programme of the European Com...",[]


In [315]:
for i in rec_system_copy["company"]:
    print(i)
    break

['Pixar Animation Studios']


In [316]:
rec_system["tags"] =  rec_system["overview"] + " " + rec_system["name_genre"]  + " " + rec_system["tagline"] + rec_system["company"] + rec_system["collection"] #+ rec_system["actors_names"] + rec_system["director_names"]+

In [317]:
rec_system

,title,overview,name_genre,actors_names,director_names,tagline,company,collection,tags
0,Toy Story,"Led by Woody, Andy's toys live happily in his ...","Comedy,Animation,Family","Sarah Freeman,Tim Allen,Annie Potts,Penn Jille...",John Lasseter,[],Pixar Animation Studios,Toy Story Collection,"Led by Woody, Andy's toys live happily in his ..."
1,Jumanji,When siblings Judy and Peter discover an encha...,"Fantasy,Adventure,Family","Malcolm Stewart,David Alan Grier,Bradley Pierc...",Joe Johnston,Roll the dice and unleash the excitement!,"Interscope Communications,TriStar Pictures,Tei...",[],When siblings Judy and Peter discover an encha...
2,Grumpier Old Men,A family wedding reignites the ancient feud be...,"Romance,Comedy","Jack Lemmon,Burgess Meredith,Walter Matthau,An...",Howard Deutch,Still Yelling. Still Fighting. Still Ready for...,"Warner Bros.,Lancaster Gate",Grumpy Old Men Collection,A family wedding reignites the ancient feud be...
3,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...","Comedy,Drama,Romance","Angela Bassett,Mykelti Williamson,Lela Rochon,...",Forest Whitaker,Friends are the people who let you be yourself...,Twentieth Century Fox Film Corporation,[],"Cheated on, mistreated and stepped on, the wom..."
4,Father of the Bride Part II,Just when George Banks has recovered from his ...,Comedy,"Kimberly Williams-Paisley,Lori Alan,BD Wong,Eu...",Charles Shyer,Just When His World Is Back To Normal... He's ...,"Sandollar Productions,Touchstone Pictures",Father of the Bride Collection,Just when George Banks has recovered from his ...
...,...,...,...,...,...,...,...,...,...
19995,"After Fall, Winter","WINTER is a dangerous, sexy, poignant and at t...","Romance,Drama","Deborah Twiss,Lizzie Brocheré,Eric Schaeffer,H...",Eric Schaeffer,[],"[,]",[],"WINTER is a dangerous, sexy, poignant and at t..."
19996,Violeta Went to Heaven,A portrait of famed Chilean singer and folklor...,Drama,"Thomas Durand,Luis Machín,Patricio Ossa,Christ...",Andrés Wood,Creation is a bird without a flight plan,Instituto Nacional de Cine y Artes Audiovisual...,[],A portrait of famed Chilean singer and folklor...
19997,Rebellion,Dissidents in a French colony attack a police ...,"Action,Drama,History","Malik Zidi,Daniel Martin,Jean-Philippe Puymart...",Mathieu Kassovitz,[],"Studio 37,Nord-Ouest Productions,UGC Images,Fr...",[],Dissidents in a French colony attack a police ...
19998,Versailles,A young mother Nina and her son Enzo find them...,Drama,"Judith Chemla,Max Baissette de Malglaive,Phili...",Pierre Schöller,[],"Procirep,Media Programme of the European Commu...",[],A young mother Nina and her son Enzo find them...


In [318]:
rec_system["tags"] = rec_system["tags"].apply(lambda x: "".join(x))

In [319]:
rec_system

,title,overview,name_genre,actors_names,director_names,tagline,company,collection,tags
0,Toy Story,"Led by Woody, Andy's toys live happily in his ...","Comedy,Animation,Family","Sarah Freeman,Tim Allen,Annie Potts,Penn Jille...",John Lasseter,[],Pixar Animation Studios,Toy Story Collection,"Led by Woody, Andy's toys live happily in his ..."
1,Jumanji,When siblings Judy and Peter discover an encha...,"Fantasy,Adventure,Family","Malcolm Stewart,David Alan Grier,Bradley Pierc...",Joe Johnston,Roll the dice and unleash the excitement!,"Interscope Communications,TriStar Pictures,Tei...",[],When siblings Judy and Peter discover an encha...
2,Grumpier Old Men,A family wedding reignites the ancient feud be...,"Romance,Comedy","Jack Lemmon,Burgess Meredith,Walter Matthau,An...",Howard Deutch,Still Yelling. Still Fighting. Still Ready for...,"Warner Bros.,Lancaster Gate",Grumpy Old Men Collection,A family wedding reignites the ancient feud be...
3,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...","Comedy,Drama,Romance","Angela Bassett,Mykelti Williamson,Lela Rochon,...",Forest Whitaker,Friends are the people who let you be yourself...,Twentieth Century Fox Film Corporation,[],"Cheated on, mistreated and stepped on, the wom..."
4,Father of the Bride Part II,Just when George Banks has recovered from his ...,Comedy,"Kimberly Williams-Paisley,Lori Alan,BD Wong,Eu...",Charles Shyer,Just When His World Is Back To Normal... He's ...,"Sandollar Productions,Touchstone Pictures",Father of the Bride Collection,Just when George Banks has recovered from his ...
...,...,...,...,...,...,...,...,...,...
19995,"After Fall, Winter","WINTER is a dangerous, sexy, poignant and at t...","Romance,Drama","Deborah Twiss,Lizzie Brocheré,Eric Schaeffer,H...",Eric Schaeffer,[],"[,]",[],"WINTER is a dangerous, sexy, poignant and at t..."
19996,Violeta Went to Heaven,A portrait of famed Chilean singer and folklor...,Drama,"Thomas Durand,Luis Machín,Patricio Ossa,Christ...",Andrés Wood,Creation is a bird without a flight plan,Instituto Nacional de Cine y Artes Audiovisual...,[],A portrait of famed Chilean singer and folklor...
19997,Rebellion,Dissidents in a French colony attack a police ...,"Action,Drama,History","Malik Zidi,Daniel Martin,Jean-Philippe Puymart...",Mathieu Kassovitz,[],"Studio 37,Nord-Ouest Productions,UGC Images,Fr...",[],Dissidents in a French colony attack a police ...
19998,Versailles,A young mother Nina and her son Enzo find them...,Drama,"Judith Chemla,Max Baissette de Malglaive,Phili...",Pierre Schöller,[],"Procirep,Media Programme of the European Commu...",[],A young mother Nina and her son Enzo find them...


In [320]:
# max_features: maxima cantidad de palabras a vectorizar
# cv = CountVectorizer(max_features=5000, stop_words="english")
tv = TfidfVectorizer(max_features=5000, stop_words="english")

In [321]:
vector = tv.fit_transform(rec_system["title"]).toarray()

In [322]:
tv.vocabulary_ # Chequeo palabras que se repitieron

{'toy': 4508,
 'story': 4219,
 'old': 3036,
 'men': 2739,
 'waiting': 4752,
 'father': 1550,
 'bride': 587,
 'ii': 2139,
 'heat': 1986,
 'sabrina': 3749,
 'tom': 4475,
 'huck': 2096,
 'sudden': 4264,
 'death': 1156,
 'american': 174,
 'president': 3411,
 'dracula': 1308,
 'dead': 1147,
 'loving': 2580,
 'balto': 338,
 'nixon': 2982,
 'island': 2206,
 'casino': 743,
 'sense': 3860,
 'rooms': 3701,
 'ace': 94,
 'ventura': 4682,
 'nature': 2934,
 'calls': 684,
 'money': 2837,
 'train': 4523,
 'assassins': 263,
 'powder': 3397,
 'leaving': 2459,
 'las': 2431,
 'vegas': 4674,
 'othello': 3073,
 'persuasion': 3200,
 'city': 861,
 'lost': 2566,
 'children': 821,
 'shanghai': 3901,
 'dangerous': 1119,
 'minds': 2790,
 'monkeys': 2840,
 'wings': 4884,
 'courage': 1012,
 'babe': 309,
 'man': 2639,
 'walking': 4759,
 'sea': 3832,
 'time': 4459,
 'takes': 4334,
 'beloved': 423,
 'country': 1008,
 'richard': 3634,
 'iii': 2140,
 'mortal': 2861,
 'kombat': 2392,
 'die': 1230,
 'make': 2628,
 'quilt'

In [323]:
cosine_sim = linear_kernel(vector,vector)

In [324]:
indices = pd.Series(rec_system.index, index=rec_system["title"]).drop_duplicates()

In [325]:
# similarity = cosine_similarity(vector)

In [326]:
def recommend(movie,cosine_sim=cosine_sim):
    movie = movie.title()
    index = indices[movie]
    sim_scores=list(enumerate(cosine_sim[index]))
    distances = sorted(sim_scores,reverse= True,key=lambda x: x[1])
    
    for i in distances[1:6]:
        # print(i[0])
        print(rec_system.iloc[i[0]].title)
recommend("toy story")

Toy Story 2
Toy Story 3
The Toy
The Toy
The Christmas Toy


In [327]:
print(tracemalloc.get_traced_memory())
tracemalloc.stop()

(4084655133, 4087198652)
